In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from os import listdir

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
data_list_model2c = listdir('/content/drive/My Drive/masterClass/tf2.0/covidDetection/train')
data_list_model2c


['COVID', 'bacterial_viral']

In [11]:
DATASET_PATH_model2c  = '/content/drive/My Drive/masterClass/tf2.0/covidDetection/train'
train_dir_model2c =  '/content/drive/My Drive/masterClass/tf2.0/covidDetection/train'
test_dir_model2c =  '/content/drive/My Drive/masterClass/tf2.0/covidDetection/test'

In [12]:
IMAGE_SIZE    = (150, 150)
NUM_CLASSES   = len(data_list_model2c)
BATCH_SIZE    = 10  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS    = 100
LEARNING_RATE = 0.0001

In [14]:
#Train datagen here is a preprocessor
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=50,
                                   featurewise_center = True,
                                   featurewise_std_normalization = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.25,
                                   zoom_range=0.1,
                                   zca_whitening = True,
                                   channel_shift_range = 20,
                                   horizontal_flip = True ,
                                   vertical_flip = True ,
                                   fill_mode='constant' )

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:342: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


In [15]:
train_batches = train_datagen.flow_from_directory(DATASET_PATH_model2c,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  subset = "training",
                                                  seed=42,
                                                  class_mode="categorical",)

Found 175 images belonging to 2 classes.


In [16]:
valid_batches = train_datagen.flow_from_directory(DATASET_PATH_model2c,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  subset = "validation",
                                                  seed=42,
                                                  class_mode="categorical",)

Found 0 images belonging to 2 classes.


In [17]:
vgg16_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(150,150,3)))

In [18]:
for layer in vgg16_model.layers[:-4]:
    layer.trainable = False
model2c = tf.keras.Sequential()
model2c.add(vgg16_model)
model2c.add(Flatten())
model2c.add(Dense(64, activation='relu'))
model2c.add(layers.Dense(2, activation='softmax'))

In [19]:
model2c.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), metrics=['accuracy'])

In [20]:
STEP_SIZE_TRAIN=train_batches.n//train_batches.batch_size
STEP_SIZE_VALID=valid_batches.n//valid_batches.batch_size

In [21]:
model2c.fit_generator(train_batches, steps_per_epoch =STEP_SIZE_TRAIN, epochs= 5, verbose=True)

Instructions for updating:
Please use Model.fit, which supports generators.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5
17/17 [==============================] - 51s 3s/step - loss: 0.7243 - accuracy: 0.5212
Epoch 2/5
17/17 [==============================] - 43s 3s/step - loss: 0.3987 - accuracy: 0.8303
Epoch 3/5
17/17 [==============================] - 43s 3s/step - loss: 0.2474 - accuracy: 0.8727
Epoch 4/5
17/17 [==============================] - 44s 3s/step - loss: 0.2811 - accuracy: 0.8848
Epoch 5/5
17/17 [==============================] - 43s 3s/step - loss: 0.1014 - accuracy: 0.9818


In [22]:
Train_DataGeneration = ImageDataGenerator(rescale=1. / 255)
TrainBatch = Train_DataGeneration.flow_from_directory(train_dir_model2c, target_size=IMAGE_SIZE, batch_size=1, shuffle=False, seed=42, class_mode="categorical")

TrainBatch.reset()
trainResult = model2c.evaluate_generator(TrainBatch, steps = len(TrainBatch), use_multiprocessing = False, verbose = 1, workers=1)
print('Train loss:' , trainResult[0], 'Train accuracy: ', trainResult[1])

prediction = model2c.predict_generator(TrainBatch, steps = len(TrainBatch))
Predicted_class = np.argmax(prediction, axis=1)

print(confusion_matrix(TrainBatch.classes, Predicted_class))

Found 175 images belonging to 2 classes.
Instructions for updating:
Please use Model.evaluate, which supports generators.
175/175 [==============================] - 43s 245ms/step - loss: 0.1059 - accuracy: 0.9600
Train loss: 0.10593054443597794 Train accuracy:  0.9599999785423279
Instructions for updating:
Please use Model.predict, which supports generators.
[[75  0]
 [ 7 93]]


In [23]:
## Test accuracy
Test_DataGeneration = ImageDataGenerator(rescale=1. / 255)
TestingBatch = Test_DataGeneration.flow_from_directory(test_dir_model2c, target_size=IMAGE_SIZE, batch_size=1, shuffle=False, seed=42, class_mode="categorical")

TestingBatch.reset()
testResult = model2c.evaluate_generator(TestingBatch, steps = len(TestingBatch), use_multiprocessing = False, verbose = 1, workers=1)
print('Test loss:' , testResult[0], 'Test accuracy: ', testResult[1])

prediction = model2c.predict_generator(TestingBatch, steps = len(TestingBatch))
Predicted_class = np.argmax(prediction, axis=1)

print(confusion_matrix(TestingBatch.classes, Predicted_class))

Found 50 images belonging to 2 classes.
50/50 [==============================] - 14s 289ms/step - loss: 0.0292 - accuracy: 1.0000
Test loss: 0.02915564738214016 Test accuracy:  1.0
[[25  0]
 [ 0 25]]
